In [4]:
import sys 
sys.path.append('../library/')

import cython
import numpy as np
import timeit

import numpy as np
from cython.parallel import prange

import cyrand

%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [5]:
%%cython
def fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a+b, a
    return a

In [6]:
X = np.array([1, 1, 1, 2, 2, 2, 3, 3, 3], dtype=np.int64) - 1
Y = np.array([1, 1, 2, 2, 2, 2, 3, 3, 4], dtype=np.int64) - 1

In [7]:
Y

array([0, 0, 1, 1, 1, 1, 2, 2, 3])

In [26]:
%%cython --annotate

import numpy as np
cimport numpy as np
from cython.parallel cimport prange

cpdef binomial(long n, long k):
    cdef long[:] C = np.zeros(k+1, dtype=np.int64)
    C[0] = 1        # nC0 is 1 
    cdef long i, j
    for i in range(1, n + 1):
        #print("i: ", i)
        for j in range(min(i, k), 0, -1):
            #print("j: ", j)
            C[j] = C[j] + C[j-1]
    return C[k]

cpdef randi(long[:] x, long[:] y):
    cdef long n = x.shape[0]
    cdef int nx = max(x) + 1
    cdef int ny = max(y) + 1
    cdef long[:,:] mat = np.zeros((nx, ny), dtype=np.int64)
    cdef int i, j
    cdef double S1=0.
    cdef double S2=0.
    cdef double S3=0.

    for i in range(x.shape[0]):
        mat[x[i], y[i]] += 1
        
    ai = np.sum(mat, axis=1)
    bj = np.sum(mat, axis=0)
    
    for i in range(nx):
        for j in range(ny):
            S1 += binomial(mat[i, j], 2)
            
    for i in range(nx):
        S2 += binomial(ai[i], 2)
            
    for j in range(ny):
        S3 += binomial(bj[j], 2)
             
    bit = (S2 * S3) / binomial(n, 2)
    cdef double ri = (S1 - bit) / (0.5 * (S2 + S3) - bit)
    
    if np.isclose(ri, 0., rtol=0.1, atol=0.1):
        #print("Close to zero")
        ri = 0.
        
    assert 0. <= ri <= 1, 'Value is out of bounds! %f' % ri
    return ri

cpdef prandi(long[:, :] x, long[:, :] y):
    """
    :param lst: List of cluster assignments
    :return:
    """
    cdef Py_ssize_t i
    cdef double[:] res = np.zeros(x.shape[0])
    assert x.shape[0] == y.shape[0], 'Arrays must be same length!'
    assert x.shape[1] == y.shape[1], 'Arrays must be same length!'
    for i in prange(x.shape[0], nogil=True):
        res[i] = 0.
    return np.array(res)

In [27]:
randi(X, Y)

TypeError: a bytes-like object is required, not 'list'

In [ ]:
help(cyrand.ri)

In [ ]:
X.dtype

In [ ]:
Y.dtype

In [ ]:
from sklearn.metrics import adjusted_rand_score as ars

ars(X, Y)

In [ ]:
for _ in range(100):
    X = np.random.randint(low=0, high=6, size=100, dtype=np.int64)
    Y = np.random.randint(low=0, high=6, size=100, dtype=np.int64)
    
    ri = randi(X, Y)
    sk = ars(X, Y)
    
    if np.isclose(sk, 0., rtol=0.1, atol=0.1):
        #print("Close to zero")
        sk = 0.
    
    assert np.isclose(ri, sk), (ri, sk)

In [ ]:
for _ in range(100):
    xalpha = np.random.dirichlet((5, 5, 5))
    yalpha = np.random.dirichlet((5, 5, 5))
    
    X = []
    Y = []
    
    X.extend([0] * int(xalpha[0] * 150))
    Y.extend([0] * int(yalpha[0] * 150))
    
    X.extend([1] * int(xalpha[1] * 150))
    Y.extend([1] * int(yalpha[1] * 150))
    
    X.extend([2] * (150 - len(X)))
    Y.extend([2] * (150 - len(Y)))
    
    X = np.array(X)
    Y = np.array(Y)
    
    ri = randi(X, Y)
    sk = ars(X, Y)
    
    if np.isclose(sk, 0., rtol=0.1, atol=0.1):
        #print("Close to zero")
        sk = 0.
    
    assert np.isclose(ri, sk), (ri, sk)

In [28]:
# THIS ONE DOESN"T WORK!

Xs = np.zeros((100, 150), dtype=np.int64)
Ys = np.zeros((100, 150), dtype=np.int64)
for i in range(100):
    xalpha = np.random.dirichlet((5, 5, 5))
    yalpha = np.random.dirichlet((5, 5, 5))
    
    X = []
    Y = []
    
    X.extend([0] * int(xalpha[0] * 150))
    Y.extend([0] * int(yalpha[0] * 150))
    
    X.extend([1] * int(xalpha[1] * 150))
    Y.extend([1] * int(yalpha[1] * 150))
    
    X.extend([2] * (150 - len(X)))
    Y.extend([2] * (150 - len(Y)))
    
    Xs[i, :] = np.array(X)
    Ys[i, :] = np.array(Y)

res = prandi(Xs, Ys)

[100, 150, 0, 0, 0, 0, 0, 0]
[100, 150, 0, 0, 0, 0, 0, 0]
False


In [29]:
res

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
Xs.shape == Ys.shape

True

In [18]:
xalpha = np.random.dirichlet((5, 5, 5))
yalpha = np.random.dirichlet((5, 5, 5))

X = []
Y = []
    
X.extend([0] * int(xalpha[0] * 150))
Y.extend([0] * int(yalpha[0] * 150))
    
X.extend([1] * int(xalpha[1] * 150))
Y.extend([1] * int(yalpha[1] * 150))
    
X.extend([2] * (150 - len(X)))
Y.extend([2] * (150 - len(Y)))
  
X = np.array(X)
Y = np.array(Y)
    
%timeit randi(X, Y) 

118 µs ± 4.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
xalpha = np.random.dirichlet((5, 5, 5))
yalpha = np.random.dirichlet((5, 5, 5))

X = []
Y = []
    
X.extend([0] * int(xalpha[0] * 150))
Y.extend([0] * int(yalpha[0] * 150))
    
X.extend([1] * int(xalpha[1] * 150))
Y.extend([1] * int(yalpha[1] * 150))
    
X.extend([2] * (150 - len(X)))
Y.extend([2] * (150 - len(Y)))
  
X = np.array(X)
Y = np.array(Y)
    
%timeit ars(X, Y) 

388 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
